<a href="https://colab.research.google.com/github/divya-160106/Sentiment-Analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy opencv-python-headless fer nltk matplotlib openpyxl

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from fer import FER
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from google.colab import drive

drive.mount('/content/drive')
nltk.download('vader_lexicon')
detector = FER()
sia = SentimentIntensityAnalyzer()

def analyze_image_emotions(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return "Unknown"

    detector = FER()
    emotions = detector.detect_emotions(image)

    if not emotions:
        return "No face detected"
    emotion_scores = emotions[0]['emotions']
    dominant_emotion = max(emotion_scores, key=emotion_scores.get)

    if dominant_emotion == "angry" and emotion_scores["angry"] > 0.8:
        return "angry"
    elif dominant_emotion == "frustrated" and emotion_scores["frustrated"] > 0.6:
        return "frustrated"
    elif dominant_emotion == "sad" and emotion_scores["sad"] > 0.4:
        return "sad"
    elif dominant_emotion == "happy":
        return "happy"
    else:
        return "frustrated"
    return max(emotions[0]['emotions'], key=emotions[0]['emotions'].get)

def analyze_text_emotions(text):
    sentiment = sia.polarity_scores(text)
    lower_text = text.lower()

    if sentiment['compound'] >= 0.4 and sentiment['pos'] > 0.3:
        return "happy"

    elif sentiment['compound'] <= -0.05:
        if "angry" in lower_text or "furious" in lower_text or "enraged" in lower_text:
            return "angry"
        if "frustrated" in lower_text or "annoyed" in lower_text:
            return "frustrated"

        if sentiment['compound'] < -0.5 and sentiment['neg'] > 0.5:
            return "sad"
        elif sentiment['compound'] < -0.3 and sentiment['neg'] > 0.3:
            return "frustrated"
        else:
            return "sad"
    return "angry"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
def analyze_images_from_drive(folder_path):
    results = {}
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('png', 'jpg', 'jpeg')):
            image_path = os.path.join(folder_path, filename)
            emotion = analyze_image_emotions(image_path)
            if emotion:
                results[filename] = emotion
    return results

def analyze_text_from_drive(file_path):
    df = pd.read_excel(file_path)
    if 'text' not in df.columns:
        raise ValueError("Excel file must contain a 'text' column.")
    df['emotion'] = df['text'].apply(analyze_text_emotions)
    return df

In [ ]:
from collections import Counter
def analyze_single_sentence(sentence):
    return analyze_text_emotions(sentence)

def summarize_emotions(image_results, text_results):
    all_emotions = list(image_results.values()) + list(text_results['emotion'])
    emotion_count = Counter(all_emotions)
    most_common_emotion, count = emotion_count.most_common(1)[0]
    messages = {
        "angry": "Anger is bad for health. Stay calm!",
        "sad": "It's okay to feel sad. Take care of yourself!",
        "frustrated": "Take a deep breath, it's okay!",
        "happy": "Keep smiling!"
    }
    return f"Most common emotion: {most_common_emotion} ({count} times). {messages.get(most_common_emotion, 'Stay positive!')}"


In [ ]:
image_folder = '/content/drive/My Drive/Projects/NLP/Images'
text_file = '/content/drive/My Drive/Projects/NLP/test_nlp.xlsx'

image_results = analyze_images_from_drive(image_folder)
text_results = analyze_text_from_drive(text_file)
print("Image Emotions:")
print(image_results)
print("Text Emotions:")
print(text_results)

summary = summarize_emotions(image_results, text_results)
print(summary)
user_sentence = input("What are you feeling right now?  ")
user_emotion = analyze_single_sentence(user_sentence)
print(f"Emotion detected: {user_emotion}")
print("\nSummary of overall emotions:")
print(summarize_emotions(image_results, text_results))

Image Emotions:
{'Happy1.jpeg': 'happy', 'Angry1.jpeg': 'angry', 'Angry2.jpeg': 'angry', 'Frustrated.jpeg': 'frustrated', 'Happy2.jpg': 'happy'}
Text Emotions:
                                                text     emotion
0  Today has been amazing! I feel so grateful for...       happy
1                I got a promotion, this is amazing!       happy
2              I keep trying, but nothing is working       angry
3  Everything feels so heavy, and I just want to ...         sad
4  I can't stop smiling, this is the best day ever!"       happy
5       That is unfair, I'm beyond furious right now       angry
6               This is so hard, it's pissing me out  frustrated
7          I feel so empty and the worls seems blank         sad
Most common emotion: happy (5 times). Keep smiling!
What are you feeling right now?  DGM sir appreciated me today
Emotion detected: happy

Summary of overall emotions:
Most common emotion: happy (5 times). Keep smiling!
